# LLaMA-Factory QuickStart 

使用 LoRA + SFT 数据, 微调 Meta-Llama-3-8B-Instruct 在 4090 上. 

1. 原始模型直接推理
2. 自定义数据集构建
3. 基于LoRA的sft指令微调
4. 动态合并LoRA的推理
5. 批量预测和训练效果评估
6. LoRA模型合并导出
7. 一站式webui board的使用
8. API Server的启动与调用
9. 大模型主流评测 benchmark


https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md  (先看这个)

https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README.md

https://github.com/hiyouga/LLaMA-Factory/blob/main/examples/README_zh.md

https://github.com/CrazyBoyM/llama3-Chinese-chat

In [ ]:
! nvidia-smi

In [ ]:
! git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
! cd LLaMA-Factory

In [ ]:
! conda create -n llama_factory python=3.10 & conda activate llama_factory

In [ ]:
! pip install -e '.[torch,metrics]' 

# optional: torch、torch_npu、metrics、deepspeed、bitsandbytes、vllm、galore、badam、gptq、awq、aqlm、qwen、modelscope、quality

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.__version__

In [ ]:
! llamafactory-cli train -h

In [ ]:
# ! git clone https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct  # HuggingFace 需要认证

# from modelscope import snapshot_download
# model_dir = snapshot_download('LLM-Research/Meta-Llama-3-8B-Instruct')

! git clone https://www.modelscope.cn/LLM-Research/Meta-Llama-3-8B-Instruct.git

# 要检查有没有下全

如果连不上 抱抱脸 

```bash
export USE_MODELSCOPE_HUB=1 # Windows 使用 `set USE_MODELSCOPE_HUB=1`
```

将 model_name_or_path 设置为模型 ID 来加载对应的模型。在 [魔搭社区](https://modelscope.cn/models) 查看所有可用的模型，例如 LLM-Research/Meta-Llama-3-8B-Instruct。

In [ ]:
# 跑一下使用 transformer 的原始推理, 确认模型文件的完整, transformer 依赖都装好了

import transformers
import torch

# 切换为你下载的模型文件目录, 这里的demo是Llama-3-8B-Instruct
# 如果是其他模型，比如qwen，chatglm，请使用其对应的官方demo
model_id = "/media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

## 使用 Llama-Factory 进行推理(WebUI)

```bash
llamafactory-cli webchat \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --template llama3
```
等效

```bash
llamafactory-cli webchat examples/inference/llama3.yaml

这里 template 在这里找 https://github.com/hiyouga/LLaMA-Factory/tree/main/examples/inference```


http://localhost:7860/

阿里云用户需要做一下如下环境变量的配置才能正常运行gradio，然后再通过阿里云给的域名映射访问

```bash
export GRADIO_ROOT_PATH=/${JUPYTER_NAME}/proxy/7860/
```

## llamafactory-cli 基本用法

```bash
llamafactory-cli version 
```

```bash
llamafactory-cli train \ 
```

```bash
llamafactory-cli chat \ 
```
    
模型合并和导出
```bash 
llamafactory-cli export \ 
```

启动API server
```bash 
llamafactory-cli api \ 
```

使用mmlu等标准数据集做评测
```bash 
llamafactory-cli eval \ 
```

前端版本纯推理的chat页面
```bash 
llamafactory-cli webchat \ 
```


启动LlamaBoard前端页面，包含可视化训练，预测，chat，模型合并多个子页面
```bash 
llamafactory-cli webui \ 
```


参数的名称（huggingface或者modelscope上的标准定义，如“meta-llama/Meta-Llama-3-8B-Instruct”）， 或者是本地下载的绝对路径，如/media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct 
```bash 
--model_name_or_path 
```

模型问答时所使用的prompt模板，不同模型不同，请参考 https://github.com/hiyouga/LLaMA-Factory?tab=readme-ov-file#supported-models 获取不同模型的模板定义，否则会回答结果会很奇怪或导致重复生成等现象的出现。chat 版本的模型基本都需要指定，比如Meta-Llama-3-8B-Instruct的template 就是 llama3
```bash
-- template
```

**一站式webui board的使用**

启动
```bash
CUDA_VISIBLE_DEVICES=0 llamafactory-cli webui
```

如果要开启 gradio的share功能，或者修改端口号
```bash
CUDA_VISIBLE_DEVICES=0 GRADIO_SHARE=1 GRADIO_SERVER_PORT=7860 llamafactory-cli webui
```

注意: 目前webui版本只支持单机单卡和单机多卡，如果是多机多卡请使用命令行版本

##  自定义数据集构建

https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README_zh.md


`data/dataset_info.json` 包含了所有可用的数据集。如果您希望使用自定义数据集，请务必在 dataset_info.json 文件中添加数据集描述，并通过修改 dataset: 数据集名称 配置来使用数据集。

目前支持 alpaca 格式和 sharegpt 格式的数据集。

alpaca
```json
[
  {
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "output": "模型回答（必填）",
    "system": "系统提示词（选填）",
    "history": [
      ["第一轮指令（选填）", "第一轮回答（选填）"],
      ["第二轮指令（选填）", "第二轮回答（选填）"]
    ]
  }
]
```

然后在 dataset_info.json 中注册

```json
"数据集名称": {
  "file_name": "data.json",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
    "system": "system",
    "history": "history"
  }
}
```

支持 预训练 和 偏好数据集(DPO, ORPO, KTO(人类反馈 true/false), 多模态) 直接看 https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README_zh.md

sharegpt 有更多角色 human、gpt、observation、function (OpenAI 格式可以看作 sharegpt 的一种特殊形式)

预训练不支持 ShareGPT 格式

---

一个关于中文 llama3 调教的 repo (star 3.2K)
https://github.com/CrazyBoyM/llama3-Chinese-chat

llama-factory 也包含了一些微调数据
https://github.com/hiyouga/LLaMA-Factory/tree/main/data

微调数据构建的注意事项
- [Paper page - LIMA: Less Is More for Alignment](https://huggingface.co/papers/2305.11206) 样本质量远比数量重要, 而微调样本的质量主要关系如下维度:
1. **样本多样性（Sample Diversity）**：
	1. **指令多样性**：考察样本中指令的覆盖范围是否广泛，是否包含了各类任务类型、不同难度级别以及多样化的指令结构和表达方式，确保模型在微调后能应对多种复杂情境。
	2. **内容多样性**：检查样本中提供的文本内容是否涵盖了不同主题、文体、长度以及语境，以避免模型在特定领域或文本类型上过拟合，确保其具备良好的泛化能力。
2. **答案质量（Answer Quality）**：
	1. **准确性（Accuracy）**：评估答案是否准确无误地响应了给定指令和内容，是否忠实反映了任务要求，且不包含事实性错误、逻辑矛盾或语义模糊。
	2. **完备性（Completeness）**：考察答案是否全面覆盖了指令所要求的所有任务点，尤其对于多步骤或复合任务，答案应完整体现所有必要的操作结果。
	3. **简洁性与清晰度（Conciseness & Clarity）**：衡量答案是否言简意赅、表达清晰，避免冗余信息或含糊表述，确保模型在微调后生成的输出易于理解和使用。
3. **一致性（Consistency）**：
	1. **内部一致性**：检查同一指令对不同内容的处理结果是否保持一致，即模型在相似情境下应给出相似的答案。
	2. **外部一致性**：对比样本答案与已知的知识库、专家判断或公认的基准结果，确保答案符合领域共识和常识。
4. **难度适配（Difficulty Calibration）**：
	1. **难易程度分布**：分析样本集中简单、中等、复杂任务的比例，确保微调数据集包含不同难度级别的样本，有助于模型逐步提升处理复杂指令的能力。
5. **噪声控制（Noise Reduction）**：
	1. **标签错误检查**：识别并剔除标注错误或不一致的样本，确保答案与指令、内容间的映射关系正确无误。
	2. **数据清洗**：去除重复样本、无关内容或低质量文本，提升数据集的整体纯净度。


可以使用我的项目自动化从txt文本中构建 instruct 数据集
https://github.com/yoko19191/auto_instruct_data_builder

## 基于LoRA 的微调

不使用 LoRA 的话, 大概微调一个7B模型就需要60G显存, 使用 LoRA 16位精度 是 16G显存. QLoRA 4 位是 6G. 




```bash
llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --dataset alpaca_gpt4_zh,identity,adgen_local \
    --dataset_dir ./data \
    --template llama3 \
    --finetuning_type lora \
    --output_dir ./saves/LLaMA3-8B/lora/sft \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 50 \
    --warmup_steps 20 \
    --save_steps 100 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 5e-5 \
    --num_train_epochs 5.0 \
    --max_samples 1000 \
    --val_size 0.1 \
    --plot_loss \
    --fp16

```



| 参数名称                   | 参数说明                                                     |
|----------------------------|----------------------------------------------------------|
| stage                      | 当前训练的阶段，枚举值，有"sft","pt","rw","ppo"等，代表了训练的不同阶段，这里我们是有监督指令微调，所以是sft |
| do_train                   | 是否是训练模式                                             |
| dataset                    | 使用的数据集列表，所有字段都需要按上文在data_info.json里注册，多个数据集用","分隔 |
| dataset_dir                | 数据集所在目录，这里是data，也就是项目自带的data目录        |
| finetuning_type            | 微调训练的类型，枚举值，有"lora","full","freeze"等，这里使用lora |
| output_dir                 | 训练结果保存的位置                                         |
| cutoff_len                 | 训练数据集的长度截断                                       |
| per_device_train_batch_size| 每个设备上的batch size，最小是1，如果GPU 显存够大，可以适当增加 |
| fp16                       | 使用半精度混合精度训练                                     |
| max_samples                | 每个数据集采样多少数据                                     |
| val_size                   | 随机从数据集中抽取多少比例的数据作为验证集                  |

注意：精度相关的参数还有bf16 和pure_bf16，但是要注意有的老显卡，比如V100就无法支持bf16，会导致程序报错或者其他错误


训练完后就可以在设置的output_dir下看到如下内容，主要包含3部分

adapter开头的就是 LoRA保存的结果了，后续用于模型推理融合

training_loss 和trainer_log等记录了训练的过程指标

其他是训练当时各种参数的备份

## 动态合并LoRA的推理

https://github.com/hiyouga/LLaMA-Factory/blob/main/examples/inference/llama3_lora_sft.yaml


当基于LoRA的训练进程结束后，我们如果想做一下动态验证，在网页端里与新模型对话，与步骤4的原始模型直接推理相比，唯一的区别是需要通过finetuning_type参数告诉系统，我们使用了LoRA训练，然后将LoRA的模型位置通过 adapter_name_or_path参数即可


```bash
llamafactory-cli webchat \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft  \
    --template llama3 \
    --finetuning_type lora
```

```bash
llamafactory-cli chat \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft  \
    --template llama3 \
    --finetuning_type lora
```

##  批量预测和训练效果评估

当然上文中的人工交互测试，会偏感性，那有没有办法批量地预测一批数据，然后使用自动化的bleu和 rouge等常用的文本生成指标来做评估。指标计算会使用如下3个库，请先做一下pip安装

In [ ]:
! pip install jieba
! pip install rouge-chinese
! pip install nltk

```bash
llamafactory-cli train \
    --stage sft \
    --do_predict \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft  \
    --dataset alpaca_gpt4_zh,identity,adgen_local \
    --dataset_dir ./data \
    --template llama3 \
    --finetuning_type lora \
    --output_dir ./saves/LLaMA3-8B/lora/predict \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_eval_batch_size 1 \
    --max_samples 20 \
    --predict_with_generate
```

| 参数名称                   | 参数说明                                            |
|----------------------------|----------------------------------------------------------|
| do_predict                 | 现在是预测模式                                              |
| predict_with_generate      | 现在用于生成文本                                            |
| max_samples                | 每个数据集采样多少用于预测对比                              |




最后会在output_dir 输出评测结果

其中 generated_predictions.jsonl 文件 输出了要预测的数据集的原始label和模型predict的结果

predict_results.json给出了原始label和模型predict的结果，用自动计算的指标数据


这里给相关的指标做一下进一步的解释


好的，这是Markdown表格格式:

| 指标 | 含义 |
| --- | --- |
| BLEU-4 | BLEU（Bilingual Evaluation Understudy）是一种常用的用于评估机器翻译质量的指标。BLEU-4 表示四元语法 BLEU 分数，它衡量模型生成文本与参考文本之间的 n-gram 匹配程度，其中 n=4。值越高表示生成的文本与参考文本越相似，最大值为 100。 |
| predict_rouge-1 和 predict_rouge-2 | ROUGE（Recall-Oriented Understudy for Gisting Evaluation）是一种用于评估自动摘要和文本生成模型性能的指标。ROUGE-1 表示一元 ROUGE 分数，ROUGE-2 表示二元 ROUGE 分数，分别衡量模型生成文本与参考文本之间的单个词和双词序列的匹配程度。值越高表示生成的文本与参考文本越相似，最大值为 100。 |
| predict_rouge-l | ROUGE-L 衡量模型生成文本与参考文本之间最长公共子序列（Longest Common Subsequence）的匹配程度。值越高表示生成的文本与参考文本越相似，最大值为 100。 |
| predict_runtime | 预测运行时间，表示模型生成一批样本所花费的总时间。单位通常为秒。 |
| predict_samples_per_second | 每秒生成的样本数量，表示模型每秒钟能够生成的样本数量。通常用于评估模型的推理速度。 |
| predict_steps_per_second | 每秒执行的步骤数量，表示模型每秒钟能够执行的步骤数量。对于生成模型，一般指的是每秒钟执行生成操作的次数。 |

## LoRA模型合并导出

https://github.com/hiyouga/LLaMA-Factory/blob/main/examples/merge_lora/llama3_lora_sft.yaml

如果想把训练的LoRA和原始的大模型进行融合，输出一个完整的模型文件的话，可以使用如下命令。合并后的模型可以自由地像使用原始的模型一样应用到其他下游环节，当然也可以递归地继续用于训练。

```bash
llamafactory-cli export \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft  \
    --template llama3 \
    --finetuning_type lora \
    --export_dir megred-model-path \
    --export_size 2 \
    --export_device cpu \
    --export_legacy_format False
```

## API Server的启动与调用


训练好后，可能部分同学会想将模型的能力形成一个可访问的网络接口，通过API 来调用，接入到langchian或者其他下游业务中，项目也自带了这部分能力。

API 实现的标准是参考了OpenAI的相关接口协议，基于uvicorn服务框架进行开发， 使用如下的方式启动

```bash
API_PORT=8000 llamafactory-cli api \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft \
    --template llama3 \
    --finetuning_type lora
```


项目也支持了基于vllm 的推理后端，但是这里由于一些限制，需要提前将LoRA 模型进行merge，使用merge后的完整版模型目录或者训练前的模型原始目录都可。
```bash
CUDA_VISIBLE_DEVICES=0 API_PORT=8000 llamafactory-cli api \
    --model_name_or_path megred-model-path \
    --template llama3 \
    --infer_backend vllm \
    --vllm_enforce_eager
```


服务启动后，即可按照openai 的API 进行远程访问，主要的区别就是替换 其中的base_url，指向所部署的机器url和端口号即可。

```python
import os
from openai import OpenAI
from transformers.utils.versions import require_version

require_version("openai>=1.5.0", "To fix: pip install openai>=1.5.0")

if __name__ == '__main__':
    # change to your custom port
    port = 8000
    client = OpenAI(
        api_key="0",
        base_url="http://localhost:{}/v1".format(os.environ.get("API_PORT", 8000)),
    )
    messages = []
    messages.append({"role": "user", "content": "hello, where is USA"})
    result = client.chat.completions.create(messages=messages, model="test")
    print(result.choices[0].message)

```

## 对模型进行量化 (GGUF, GPTQ, AWQ, EXL2)


https://huggingface.co/blog/zh/hf-bitsandbytes-integration

Safetensors 和 PyTorch 二进制文件是原始 float16 模型文件的示例。这些文件主要用于持续微调目的。

GGML 与 GGUF 指代同一概念，其中 GGUF 作为较新版本，融入了关于模型的额外数据。此项增强功能支持多种架构，并包含提示模板。GGUF 可仅在 CPU 上执行，或部分/完全卸载至 GPU。借助 K 量化技术，GGUF 的位宽可从 2 位扩展至 8 位。


先前，GPTQ 作为一种仅针对 GPU 优化的量化方法，但已被 AWQ 超越，后者速度约快两倍。该领域最新进展是 EXL2，其性能更佳。通常，这些量化方法采用 4 位实现

我们可能需要把微调后的进行量化和模型转为 GGUF 格式, 方便 Ollama llama.cpp 的部署推理. 

用这个可以快速吧HuggingFace 上的模型改为GGUF格式
https://huggingface.co/spaces/ggml-org/gguf-my-repo


或者使用 llama.cpp 进行量化

(没写完)

## 进阶-大模型主流评测 benchmark

虽然大部分同学的主流需求是定制一个下游的垂直模型，但是在部分场景下，也可能有同学会使用本项目来做更高要求的模型训练，用于大模型刷榜单等，比如用于评测mmlu等任务。当然这类评测同样可以用于评估大模型二次微调之后，对于原来的通用知识的泛化能力是否有所下降。（因为一个好的微调，尽量是在具备垂直领域知识的同时，也保留了原始的通用能力）

本项目提供了mmlu，cmmlu, ceval三个常见数据集的自动评测脚本，按如下方式进行调用即可


如果是Chat model 
```bash
CUDA_VISIBLE_DEVICES=0 llamafactory-cli eval \
--model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
--template llama3 \
--task mmlu \
--split validation \
--lang en \
--n_shot 5 \
--batch_size 1
```


可能的输出如下, 具体任务的指标定义请参考mmlu，cmmlu, ceval等任务原始的相关资料, 和llama3的官方报告基本一致

```text
Average: 63.64                                                                                                                                     
           STEM: 50.83
Social Sciences: 76.31
     Humanities: 56.63
          Other: 73.31

```

如果是base版本的模型，template改为fewshot即可

```bash
CUDA_VISIBLE_DEVICES=0 llamafactory-cli eval \
--model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B \
--template fewshot \
--task mmlu \
--split validation \
--lang en \
--n_shot 5 \
--batch_size 1
```